In [4]:
# Imports 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
sns.set()
sns.set_context('talk')

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 104)
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest
# Note: the statsmodels import may print out a 'FutureWarning'. Thats fine.

/var/folders/p7/ydpytwds077_jrkmfgmnwl1m0000gn/T/ipykernel_4767/3938033941.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


This file should be used to collect and scrape our code and stuff. thanks!

In [6]:
# for BMI csv, region/world
bmi = pd.read_csv("../datasets/bmi_country.csv")

In [35]:
bmi.head() # basically just checking if it is properly uploaded and works

,Country/Region/World,ISO,Sex,Year,Mean BMI,Mean BMI lower 95% uncertainty interval,Mean BMI upper 95% uncertainty interval,Prevalence of BMI>=30 kg/m² (obesity),Prevalence of BMI>=30 kg/m² lower 95% uncertainty interval,Prevalence of BMI>=30 kg/m² upper 95% uncertainty interval,Prevalence of BMI>=35 kg/m² (severe obesity),Prevalence of BMI>=35 kg/m² lower 95% uncertainty interval,Prevalence of BMI>=35 kg/m² upper 95% uncertainty interval,Prevalence of BMI<18.5 kg/m² (underweight),Prevalence of BMI<18.5 kg/m² lower 95% uncertainty interval,Prevalence of BMI<18.5 kg/m² upper 95% uncertainty interval,Prevalence of BMI 18.5 kg/m² to <20 kg/m²,Prevalence of BMI 18.5 kg/m² to <20 kg/m² lower 95% uncertainty interval,Prevalence of BMI 18.5 kg/m² to <20 kg/m² upper 95% uncertainty interval,Prevalence of BMI 20 kg/m² to <25 kg/m²,Prevalence of BMI 20 kg/m² to <25 kg/m² lower 95% uncertainty interval,Prevalence of BMI 20 kg/m² to <25 kg/m² upper 95% uncertainty interval,Prevalence of BMI 25 kg/m² to <30 kg/m²,Prevalence of BMI 25 kg/m² to <30 kg/m² lower 95% uncertainty interval,Prevalence of BMI 25 kg/m² to <30 kg/m² upper 95% uncertainty interval,Prevalence of BMI 30 kg/m² to <35 kg/m²,Prevalence of BMI 30 kg/m² to <35 kg/m² lower 95% uncertainty interval,Prevalence of BMI 30 kg/m² to <35 kg/m² upper 95% uncertainty interval,Prevalence of BMI 35 kg/m² to <40 kg/m²,Prevalence of BMI 35 kg/m² to <40 kg/m² lower 95% uncertainty interval,Prevalence of BMI 35 kg/m² to <40 kg/m² upper 95% uncertainty interval,Prevalence of BMI >=40 kg/m²(morbid obesity),Prevalence of BMI >=40 kg/m² lower 95% uncertainty interval,Prevalence of BMI >=40 kg/m² upper 95% uncertainty interval
0,Afghanistan,AFG,Men,1975,18.999440,16.331119,21.763077,0.002078,0.000384,0.006515,0.000034,0.000002,0.000158,0.284239,0.147565,0.421922,0.234271,0.134912,0.347236,0.428146,0.312880,0.551022,0.051266,0.021452,0.096419,0.002043,0.000351,0.006493,0.000027,6.663263e-07,0.000138,0.000008,5.054664e-08,0.000047
1,Afghanistan,AFG,Men,1976,19.105518,16.500077,21.801109,0.002237,0.000437,0.006820,0.000038,0.000002,0.000168,0.281990,0.148634,0.417047,0.232208,0.136020,0.340563,0.430523,0.317865,0.549570,0.053042,0.022928,0.097881,0.002199,0.000403,0.006779,0.000030,8.525116e-07,0.000149,0.000009,6.816061e-08,0.000049
2,Afghanistan,AFG,Men,1977,19.212080,16.646507,21.842021,0.002410,0.000497,0.007179,0.000042,0.000003,0.000185,0.279694,0.149224,0.412887,0.230125,0.136671,0.335139,0.432881,0.322810,0.549225,0.054890,0.024613,0.098869,0.002368,0.000454,0.007144,0.000033,1.080451e-06,0.000163,0.000009,9.255251e-08,0.000053
3,Afghanistan,AFG,Men,1978,19.319053,16.786899,21.876516,0.002599,0.000559,0.007567,0.000047,0.000003,0.000202,0.277336,0.150410,0.407314,0.228031,0.137231,0.330865,0.435219,0.327795,0.548325,0.056815,0.026318,0.100453,0.002551,0.000519,0.007540,0.000037,1.364412e-06,0.000178,0.000010,1.262377e-07,0.000057
4,Afghanistan,AFG,Men,1979,19.425952,16.950002,21.930634,0.002804,0.000636,0.008026,0.000053,0.000004,0.000220,0.274931,0.151589,0.403758,0.225927,0.137064,0.327193,0.437522,0.332613,0.547925,0.058816,0.028116,0.102426,0.002751,0.000581,0.007971,0.000041,1.698723e-06,0.000193,0.000011,1.576738e-07,0.000062


In [7]:
# CLEANING UP TABLE
bmi = bmi[bmi['Year'] >= 2000] # updating it so that it is only in 21st century
if 'ISO' in bmi.columns: # deletes NaN column that wasn't used
    del bmi['ISO']
bmi.head()

,Country/Region/World,Sex,Year,Mean BMI,Mean BMI lower 95% uncertainty interval,Mean BMI upper 95% uncertainty interval,Prevalence of BMI>=30 kg/m² (obesity),Prevalence of BMI>=30 kg/m² lower 95% uncertainty interval,Prevalence of BMI>=30 kg/m² upper 95% uncertainty interval,Prevalence of BMI>=35 kg/m² (severe obesity),Prevalence of BMI>=35 kg/m² lower 95% uncertainty interval,Prevalence of BMI>=35 kg/m² upper 95% uncertainty interval,Prevalence of BMI<18.5 kg/m² (underweight),Prevalence of BMI<18.5 kg/m² lower 95% uncertainty interval,Prevalence of BMI<18.5 kg/m² upper 95% uncertainty interval,Prevalence of BMI 18.5 kg/m² to <20 kg/m²,Prevalence of BMI 18.5 kg/m² to <20 kg/m² lower 95% uncertainty interval,Prevalence of BMI 18.5 kg/m² to <20 kg/m² upper 95% uncertainty interval,Prevalence of BMI 20 kg/m² to <25 kg/m²,Prevalence of BMI 20 kg/m² to <25 kg/m² lower 95% uncertainty interval,Prevalence of BMI 20 kg/m² to <25 kg/m² upper 95% uncertainty interval,Prevalence of BMI 25 kg/m² to <30 kg/m²,Prevalence of BMI 25 kg/m² to <30 kg/m² lower 95% uncertainty interval,Prevalence of BMI 25 kg/m² to <30 kg/m² upper 95% uncertainty interval,Prevalence of BMI 30 kg/m² to <35 kg/m²,Prevalence of BMI 30 kg/m² to <35 kg/m² lower 95% uncertainty interval,Prevalence of BMI 30 kg/m² to <35 kg/m² upper 95% uncertainty interval,Prevalence of BMI 35 kg/m² to <40 kg/m²,Prevalence of BMI 35 kg/m² to <40 kg/m² lower 95% uncertainty interval,Prevalence of BMI 35 kg/m² to <40 kg/m² upper 95% uncertainty interval,Prevalence of BMI >=40 kg/m²(morbid obesity),Prevalence of BMI >=40 kg/m² lower 95% uncertainty interval,Prevalence of BMI >=40 kg/m² upper 95% uncertainty interval
25,Afghanistan,Men,2000,21.369506,19.153423,23.559675,0.012297,0.004712,0.026176,0.000582,0.000112,0.001755,0.218857,0.116488,0.329003,0.181228,0.115501,0.257605,0.474904,0.392218,0.562829,0.112715,0.074823,0.161792,0.011715,0.004119,0.025631,0.000440,0.000053,0.001507,0.000142,0.000008,0.000623
26,Afghanistan,Men,2001,21.456846,19.226323,23.641168,0.013129,0.005102,0.027750,0.000649,0.000128,0.001936,0.215744,0.114774,0.325905,0.178977,0.113907,0.254904,0.476190,0.394457,0.562832,0.115959,0.077215,0.165228,0.012479,0.004480,0.027253,0.000489,0.000060,0.001679,0.000160,0.000010,0.000694
27,Afghanistan,Men,2002,21.543463,19.301239,23.727827,0.014011,0.005522,0.029456,0.000724,0.000145,0.002104,0.212595,0.112654,0.322228,0.176702,0.112743,0.251709,0.477428,0.396318,0.563393,0.119263,0.080114,0.168613,0.013287,0.004841,0.028726,0.000544,0.000070,0.001836,0.000181,0.000011,0.000769
28,Afghanistan,Men,2003,21.628227,19.394877,23.830242,0.014947,0.006002,0.031181,0.000807,0.000167,0.002341,0.209419,0.110636,0.318674,0.174411,0.110997,0.248713,0.478601,0.397541,0.564361,0.122622,0.082665,0.173029,0.014140,0.005196,0.030535,0.000604,0.000080,0.002018,0.000204,0.000013,0.000860
29,Afghanistan,Men,2004,21.710447,19.476779,23.912341,0.015937,0.006489,0.032795,0.000900,0.000187,0.002584,0.206212,0.108702,0.315840,0.172098,0.109884,0.246124,0.479721,0.399063,0.564695,0.126031,0.085312,0.177013,0.015038,0.005581,0.032024,0.000670,0.000092,0.002221,0.000230,0.000015,0.000959


Next up, standardizing the table, with sex and relevant BMI information

In [8]:
def standardize_gender(str_in): 
    try:
        if pd.notna(str_in):
            str_in = str_in.lower().strip()

            
            if 'women' in str_in:
                return str_in.replace('women', 'F')
            elif 'men' in str_in:
                return str_in.replace('men', 'M')
            else:
                return np.nan
        else:
            return np.nan
    except Exception as e:
        print(f"An error occurred: {e}")
        return np.nan

In [9]:
bmi['Sex'] = bmi['Sex'].apply(standardize_gender)

In [10]:
# cleans up uncertainty interval, we can add back if needed for EDA but not sure
for col in bmi.columns:
    if 'uncertainty interval' in col:
        del bmi[col]
bmi

,Country/Region/World,Sex,Year,Mean BMI,Prevalence of BMI>=30 kg/m² (obesity),Prevalence of BMI>=35 kg/m² (severe obesity),Prevalence of BMI<18.5 kg/m² (underweight),Prevalence of BMI 18.5 kg/m² to <20 kg/m²,Prevalence of BMI 20 kg/m² to <25 kg/m²,Prevalence of BMI 25 kg/m² to <30 kg/m²,Prevalence of BMI 30 kg/m² to <35 kg/m²,Prevalence of BMI 35 kg/m² to <40 kg/m²,Prevalence of BMI >=40 kg/m²(morbid obesity)
25,Afghanistan,M,2000,21.369506,0.012297,0.000582,0.218857,0.181228,0.474904,0.112715,0.011715,0.000440,0.000142
26,Afghanistan,M,2001,21.456846,0.013129,0.000649,0.215744,0.178977,0.476190,0.115959,0.012479,0.000489,0.000160
27,Afghanistan,M,2002,21.543463,0.014011,0.000724,0.212595,0.176702,0.477428,0.119263,0.013287,0.000544,0.000181
28,Afghanistan,M,2003,21.628227,0.014947,0.000807,0.209419,0.174411,0.478601,0.122622,0.014140,0.000604,0.000204
29,Afghanistan,M,2004,21.710447,0.015937,0.000900,0.206212,0.172098,0.479721,0.126031,0.015038,0.000670,0.000230
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,Zimbabwe,F,2012,25.284840,0.244698,0.084217,0.049484,0.072888,0.355026,0.277904,0.160482,0.055707,0.028509
16796,Zimbabwe,F,2013,25.319948,0.249242,0.086849,0.048700,0.071592,0.351572,0.278894,0.162394,0.057094,0.029754
16797,Zimbabwe,F,2014,25.356222,0.253836,0.089553,0.047933,0.070315,0.348091,0.279826,0.164283,0.058500,0.031054
16798,Zimbabwe,F,2015,25.393409,0.258485,0.092343,0.047186,0.069068,0.344566,0.280695,0.166141,0.059935,0.032408
